## Benchmarking with Argo Worfklows & Vegeta

In this notebook we will dive into how you can run bench marking with batch processing with Argo Workflows, Seldon Core and Vegeta.

Dependencies:

* Seldon core installed as per the docs with Istio as an ingress 
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Setup

### Install Seldon Core
Use the notebook to [set-up Seldon Core with Ambassador or Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

Note: If running with KIND you need to make sure do follow [these steps](https://github.com/argoproj/argo/issues/2376#issuecomment-595593237) as workaround to the `/.../docker.sock` known issue.


### Install Argo Workflows
You can follow the instructions from the official [Argo Workflows Documentation](https://github.com/argoproj/argo#quickstart).

Download the right CLi for your environment following the documentation (https://github.com/argoproj/argo-workflows/releases/tag/v3.0.8)

You also need to make sure that argo has permissions to create seldon deployments - for this you can just create a default-admin rolebinding as follows:

In [660]:
!kubectl create namespace argo || echo "namespace already created"
!kubectl apply -n argo -f https://raw.githubusercontent.com/argoproj/argo/stable/manifests/install.yaml
!kubectl rollout status -n argo deployment/argo-server
!kubectl rollout status -n argo deployment/workflow-controller

Error from server (AlreadyExists): namespaces "argo" already exists
namespace already created
customresourcedefinition.apiextensions.k8s.io/clusterworkflowtemplates.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/cronworkflows.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workfloweventbindings.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workflows.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workflowtemplates.argoproj.io created
serviceaccount/argo created
serviceaccount/argo-server created
role.rbac.authorization.k8s.io/argo-role created
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-admin created
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-edit created
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-view created
clusterrole.rbac.authorization.k8s.io/argo-cluster-role created
clusterrole.rbac.authorization.k8s.io/argo-server-cluster-role created
rolebinding.rbac.authorization.k

Set up the RBAC so the argo workflow is able to create seldon deployments.

In [661]:
# RBAC for argo job to be able to access it's information as it progresses
!kubectl create rolebinding argo-default-admin --clusterrole=admin --serviceaccount=argo:default -n argo
# RBAC for argo workflow to be able to create seldon resources
!kubectl create rolebinding argo-seldon-workflow --clusterrole=seldon-manager-role-seldon-system --serviceaccount=argo:default -n argo

rolebinding.rbac.authorization.k8s.io/argo-default-admin created
rolebinding.rbac.authorization.k8s.io/argo-seldon-workflow created


Set up the configmap in order for it to work in KIND and other environments where Docker may not be thr main runtime (see https://github.com/argoproj/argo-workflows/issues/5243#issuecomment-792993742)

In [662]:
%%bash
kubectl apply -n argo -f - << END
apiVersion: v1
kind: ConfigMap
metadata:
  name: workflow-controller-configmap
data:
  containerRuntimeExecutor: k8sapi
END

configmap/workflow-controller-configmap configured


### Create Benchmark Argo Workflow

In order to create a benchmark, we created a simple argo workflow template so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 1 replicas and 4 cpus (with 100 max workers) to send requests.

In [664]:
!helm template seldon-benchmark-workflow helm-charts/seldon-benchmark-workflow/ \
    --set workflow.namespace=argo \
    --set workflow.name=seldon-benchmark-process \
    --set workflow.parallelism=3 \
    --set seldonDeployment.name=sklearn \
    --set seldonDeployment.replicas="1" \
    --set seldonDeployment.serverWorkers="2" \
    --set seldonDeployment.serverThreads=1 \
    --set seldonDeployment.modelUri="gs://seldon-models/sklearn/iris" \
    --set seldonDeployment.server="SKLEARN_SERVER" \
    --set seldonDeployment.apiType="rest|grpc" \
    --set seldonDeployment.requests.cpu="500Mi" \
    --set seldonDeployment.requests.limits="2000Mi" \
    --set seldonDeployment.disableOrchestrator="false" \
    --set benchmark.cpu="5" \
    --set benchmark.concurrency="1" \
    --set benchmark.duration="30s" \
    --set benchmark.rate=0 \
    --set benchmark.data='\{"data": {"ndarray": [[0\,1\,2\,3]]\}\}' \
    | argo submit -

Name:                seldon-benchmark-process
Namespace:           argo
ServiceAccount:      default
Status:              Pending
Created:             Sun Jun 27 14:30:24 +0100 (now)
Progress:            


In [665]:
!argo list -n argo

NAME                       STATUS    AGE   DURATION   PRIORITY
seldon-benchmark-process   Running   1s    1s         0


In [666]:
!argo logs -f seldon-benchmark-process -n argo

seldon-benchmark-process-635956972: [{"name": "sklearn-0", "replicas": "1", "serverWorkers": "2", "serverThreads": "1", "modelUri": "gs://seldon-models/sklearn/iris", "image": "", "server": "SKLEARN_SERVER", "apiType": "rest", "requestsCpu": "500Mi", "requestsMemory": "100Mi", "limitsCpu": "50m", "limitsMemory": "1000Mi", "benchmarkCpu": "5", "concurrency": "1", "duration": "30s", "rate": "0", "disableOrchestrator": "false", "params": "{\"name\": \"sklearn-0\", \"replicas\": \"1\", \"serverWorkers\": \"2\", \"serverThreads\": \"1\", \"modelUri\": \"gs://seldon-models/sklearn/iris\", \"image\": \"\", \"server\": \"SKLEARN_SERVER\", \"apiType\": \"rest\", \"requestsCpu\": \"500Mi\", \"requestsMemory\": \"100Mi\", \"limitsCpu\": \"50m\", \"limitsMemory\": \"1000Mi\", \"benchmarkCpu\": \"5\", \"concurrency\": \"1\", \"duration\": \"30s\", \"rate\": \"0\", \"disableOrchestrator\": \"false\"}"}, {"name": "sklearn-1", "replicas": "1", "serverWorkers": "2", "serverThreads": "1", "modelUri": "g

seldon-benchmark-process-4000812633: Waiting for deployment "sklearn-1-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-789106512: Waiting for deployment "sklearn-0-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-789106512: deployment "sklearn-0-default-0-classifier" successfully rolled out
seldon-benchmark-process-4000812633: deployment "sklearn-1-default-0-classifier" successfully rolled out
seldon-benchmark-process-345794713: {"latencies":{"total":29975520300,"mean":7300418,"50th":6839232,"90th":9876051,"95th":11144557,"99th":14635289,"max":51580500,"min":3999600},"bytes_in":{"total":751398,"mean":183},"bytes_out":{"total":139604,"mean":34},"earliest":"2021-06-27T13:31:16.2189964Z","latest":"2021-06-27T13:31:46.2223233Z","end":"2021-06-27T13:31:46.2309266Z","duration":30003326900,"wait":8603300,"requests":4106,"rate":136.85149029256485,"throughput":136.81226007916013,

In [667]:
!argo get seldon-benchmark-process -n argo

Name:                seldon-benchmark-process
Namespace:           argo
ServiceAccount:      default
Status:              Succeeded
Conditions:          
 PodRunning          False
 Completed           True
Created:             Sun Jun 27 14:30:24 +0100 (1 minute ago)
Started:             Sun Jun 27 14:30:24 +0100 (1 minute ago)
Finished:            Sun Jun 27 14:32:06 +0100 (1 second ago)
Duration:            1 minute 42 seconds
Progress:            9/9
ResourcesDuration:   3m19s*(1 cpu),3m19s*(100Mi memory)

STEP                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Process the results

We can now print the results in a consumable format.

In [668]:
def get_results(results, print_results=True):
    final = {}
    # For GHZ / grpc
    if "average" in results:
        final["mean"] = results["average"] / 1e6
        if results.get("latencyDistribution", False):
            final["50th"] = results["latencyDistribution"][-5]["latency"] / 1e6
            final["90th"] = results["latencyDistribution"][-3]["latency"] / 1e6
            final["95th"] = results["latencyDistribution"][-2]["latency"] / 1e6
            final["99th"] = results["latencyDistribution"][-1]["latency"] / 1e6
        final["throughputAchieved"] = results["rps"]
        final["success"] = results["statusCodeDistribution"].get("OK", 0)
        final["errors"] = sum(results["statusCodeDistribution"].values()) - final["success"]
    # For vegeta / rest
    else:
        final["mean"] = results["latencies"]["mean"] / 1e6
        final["50th"] = results["latencies"]["50th"] / 1e6
        final["90th"] = results["latencies"]["90th"] / 1e6
        final["95th"] = results["latencies"]["95th"] / 1e6
        final["99th"] = results["latencies"]["99th"] / 1e6
        final["throughputAchieved"] = results["throughput"]
        final["success"] = results["status_codes"].get("200", 0)
        final["errors"] = sum(results["status_codes"].values()) - final["success"]
    for k in results["params"].keys():
        final[k] = results["params"][k]
    if print_results:
        print("-----")
        print("ParamNames:", results["params"].keys())
        print("ParamNames:", results["params"].values())
        print("\tLatencies:")
        print("\t\tmean:", final["mean"], "ms")
        print("\t\t50th:", final["50th"], "ms")
        print("\t\t90th:", final["90th"], "ms")
        print("\t\t95th:", final["95th"], "ms")
        print("\t\t99th:", final["99th"], "ms")
        print("")
        print("\tRate:", str(final["throughputAchieved"]) + "/s")
        print("\tSuccess:", final["success"])
        print("\tErrors:", final["errors"])
    return final

In [669]:
import json
wf_logs = !argo logs --no-color seldon-benchmark-process -n argo
wf_bench = [json.loads(":".join(w.split(":")[1:])) for w in wf_logs if "latenc" in w]
print(wf_bench)

[{'latencies': {'total': 29975520300, 'mean': 7300418, '50th': 6839232, '90th': 9876051, '95th': 11144557, '99th': 14635289, 'max': 51580500, 'min': 3999600}, 'bytes_in': {'total': 751398, 'mean': 183}, 'bytes_out': {'total': 139604, 'mean': 34}, 'earliest': '2021-06-27T13:31:16.2189964Z', 'latest': '2021-06-27T13:31:46.2223233Z', 'end': '2021-06-27T13:31:46.2309266Z', 'duration': 30003326900, 'wait': 8603300, 'requests': 4106, 'rate': 136.85149029256485, 'throughput': 136.81226007916013, 'success': 1, 'status_codes': {'200': 4106}, 'errors': [], 'params': {'name': 'sklearn-0', 'replicas': '1', 'serverWorkers': '2', 'serverThreads': '1', 'modelUri': 'gs://seldon-models/sklearn/iris', 'image': '', 'server': 'SKLEARN_SERVER', 'apiType': 'rest', 'requestsCpu': '500Mi', 'requestsMemory': '100Mi', 'limitsCpu': '50m', 'limitsMemory': '1000Mi', 'benchmarkCpu': '5', 'concurrency': '1', 'duration': '30s', 'rate': '0', 'disableOrchestrator': 'false'}}, {'date': '2021-06-27T13:31:47Z', 'endReason

In [670]:
results = []
for w in wf_bench:
    # Prints the results in a consumable format
    results.append(get_results(w))

-----
ParamNames: dict_keys(['name', 'replicas', 'serverWorkers', 'serverThreads', 'modelUri', 'image', 'server', 'apiType', 'requestsCpu', 'requestsMemory', 'limitsCpu', 'limitsMemory', 'benchmarkCpu', 'concurrency', 'duration', 'rate', 'disableOrchestrator'])
ParamNames: dict_values(['sklearn-0', '1', '2', '1', 'gs://seldon-models/sklearn/iris', '', 'SKLEARN_SERVER', 'rest', '500Mi', '100Mi', '50m', '1000Mi', '5', '1', '30s', '0', 'false'])
	Latencies:
		mean: 7.300418 ms
		50th: 6.839232 ms
		90th: 9.876051 ms
		95th: 11.144557 ms
		99th: 14.635289 ms

	Rate: 136.81226007916013/s
	Success: 4106
	Errors: 0
-----
ParamNames: dict_keys(['name', 'replicas', 'serverWorkers', 'serverThreads', 'modelUri', 'image', 'server', 'apiType', 'requestsCpu', 'requestsMemory', 'limitsCpu', 'limitsMemory', 'benchmarkCpu', 'concurrency', 'duration', 'rate', 'disableOrchestrator'])
ParamNames: dict_values(['sklearn-1', '1', '2', '1', 'gs://seldon-models/sklearn/iris', '', 'SKLEARN_SERVER', 'grpc', '500

In [663]:
!argo delete seldon-benchmark-process -n argo || echo "Argo workflow already deleted or not exists"

Workflow 'seldon-benchmark-process' not found


In [ ]:
!kubectl delete -n argo -f https://raw.githubusercontent.com/argoproj/argo/v3.1.0/manifests/install.yaml

## Deeper Analysis
Now that we have all the parameters, we can do a deeper analysis

In [672]:
import pandas as pd

df = pd.DataFrame.from_dict(results)
df.head()

,mean,50th,90th,95th,99th,throughputAchieved,success,errors,name,replicas,...,apiType,requestsCpu,requestsMemory,limitsCpu,limitsMemory,benchmarkCpu,concurrency,duration,rate,disableOrchestrator
0,7.300418,6.839232,9.876051,11.144557,14.635289,136.81226,4106,0,sklearn-0,1,...,rest,500Mi,100Mi,50m,1000Mi,5,1,30s,0,false
1,6.674704,6.330400,8.990000,10.044700,12.267200,144.46257,4333,1,sklearn-1,1,...,grpc,500Mi,100Mi,50m,1000Mi,5,1,30s,0,false


In [ ]:
!jx step pr comment --owner=${REPO_OWNER} --repository=${REPO_NAME} --pull-request=${PULL_NUMBER} --comment="${df.to_markdown()}"